## Trabajo práctico Integrador

**Alumno:** Torresetti Lisandro

**Padrón:** 99846

**Objetivo:**

Realizar un programa que permita determiar la pose de las piezas dispuestas en la zona de trabajo de manera que el robot pueda tomarlas.

In [ ]:
# Imports
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import PIL.ExifTags
import PIL.Image
import pprint
import re
%matplotlib inline

# Paths

BLOCKS_PATH = './imagenes/img_bloques'
BLOCKS_CHALLENGE_PATH = './imagenes/img_bloques_desafio'
CALIBRATION_SET_1 = './imagenes/img_cal_set1'
CALIBRATION_SET_2 = './imagenes/img_cal_set2'

# Constants

COLOR = 0
GRAY = 1
EXTRA_IMG = 'imgCalExtr'

# Debug
IMGS_DEBUG = ['imgBloque1', 'imgBloque2', 'imgBloque3', 'imgBloque4', 'imgBloque20']

## Auxiliary Functions

In [ ]:
# Auxiliary function to make the plots
def plotter(image, title = '', imgSize = (16,9), grayScale = False):
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.show()

# Auxiliary function to load all the images from path
def loadImages(path):
    imgNames = glob(path + '/*')
    result = {}
    for imgName in imgNames:
        imgs = []
        img = cv.imread(imgName)
        imgs.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        imgs.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
        result[re.sub(r'^.*/([^.]*).*$', r'\1', imgName)] = imgs
    return result

# Creates a mask from the image and the specified position. Remember: in OpenCv the coorinates are (Y, X)
def createMask(img, position, lowerMultiplier = 6, upperMultiplier = 6):
    initialPoint, endPoint = position
    colorMean, colorStd = cv.meanStdDev(img[initialPoint[1]:endPoint[1], initialPoint[0]:endPoint[0], :])
    return cv.inRange(img, colorMean - colorStd * lowerMultiplier,  colorMean + colorStd * upperMultiplier)

## Process Images

Primero cargo las imagenes de los bloques y las de los sets de calibracion.

## Hay que rotar todas las imagenes de calibracion para que apunten en la misma direccion??

In [ ]:
imgsBlocks = loadImages(BLOCKS_PATH)
calibrationSet1 = loadImages(CALIBRATION_SET_1)
calibrationSet2 = loadImages(CALIBRATION_SET_2)

# Dado que en las imagenes de los bloques hay una imagen para realizar la calibracion extrinseca
# la guardo en una variable y la elimino del set a analizar
extrinsicCalibration = imgsBlocks[EXTRA_IMG]
del imgsBlocks[EXTRA_IMG]

# Imprimo las imagenes para verificar que este todo bien
for block in imgsBlocks.values():
    plotter(block[COLOR])

Creo una mascara con el tablero verde para poder eliminar el ruido de los otros elementos que se encuentran en la imagen. Para crearla, tomare una porcion de 'imgBloque1' y otra de 'imgBloque2' para luego hacer un OR y obtener una mascara que solo abarque el tablero verde.

In [ ]:
# Position = ((x1, y1), (x2, y2))
maskPosition1 = ((125, 280), (480, 450))
blocksMask1 = createMask(imgsBlocks['imgBloque1'][COLOR], maskPosition1)

maskPosition2 = ((125, 50), (480, 250))
blocksMask2 = createMask(imgsBlocks['imgBloque4'][COLOR], maskPosition2)
blocksMask = cv.bitwise_or(blocksMask1, blocksMask2)
blocksMask = cv.medianBlur(blocksMask, 15)

blocksWithMask = {}
for imgName, values in imgsBlocks.items():
    blocksWithMask[imgName] = cv.bitwise_and(values[COLOR], values[COLOR], mask=blocksMask)

# DEBUG
for imgName in IMGS_DEBUG:
    plotter(blocksWithMask[imgName])

## Step 1 - Intrinsic Parameters

## Step 2 - Extrinsic Calibration

## Step 3 - Search Blocks

## Step 4 - Validation

## Step 5 - Measure Blocks